In [1]:
########## Multilabel - Data Augumented - Universal sentense Encoder -  Keras Modelling ####################

In [2]:
#################################################################

In [174]:
import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Layer, Concatenate, concatenate
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.optimizers import Adam


from tensorflow.keras import backend as K

session  = tf.Session()
K.set_session(session)
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())



In [2]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd

df = pd.DataFrame(columns=['label','text'])
trainset = datasets.load_files(container_path = 'dataset_small_less_categories_v2',encoding = 'UTF-8')

def get_dataframe(lines):
#     lines = filename.splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i][0]
    #     label = label.split(",")[0]
        text = ' '.join(lines[i][1:])
        text = re.sub(',','', text)
        data.append([label, text])
    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

for k in range(len(trainset.target)):
    labelled_sent= trainset.data[k].strip().split('\r\n')
    Label=[trainset.target[k]]*len(labelled_sent)
    labelled_emb = list(zip([s for s in Label], [s for s in labelled_sent]))
    df_train = get_dataframe(labelled_emb)
    df=pd.concat([df,df_train])
    df.reset_index(inplace=True, drop=True)

In [3]:
df

,label,text
0,5,I want to travel back to my native place.
1,5,Want curfew pass for case of emergency travel.
2,5,I had started towards my home 120 km away on m...
3,5,Firstline workers facing issues as no cabs are...
4,5,Want to go back to my family place. Currently ...
...,...,...
333,4,Economic downfall will soon create job crises.
334,4,Employee are not happy with the relief funds p...
335,4,emi should be reduced and less interest rate s...
336,4,tv news channel focus more on fake news.


In [4]:
#trainset.data[0], trainset.target[0]

In [5]:
trainset

{'data': ["I want to travel back to my native place.\r\nWant curfew pass for case of emergency travel.\r\nI had started towards my home 120 km away on my car.\r\nFirstline workers facing issues as no cabs are available\r\nWant to go back to my family place. Currently staying in the hotel.\r\nWant to travel from bangalore to delhi. Any means of transport available?\r\nI haven't seen my family for a long time and they need my support. I need to be there.\r\nResiding in green zone but not able to get travel permssion even for getting essential household items.\r\nFew days back my vehicle was seized by police, now don't have any means of transport.\r\nMy visa about to get expire. Want some travel arrangements to be done to fly back to Austria.\r\nFive construction workers started walking from kerela to bihar but are stranded in karnataka border.\r\nI live in pg where its difficult to get food. When are flights and trains getting started.\r\nYou are giving permission to celebrities like Ris

In [6]:
pd.set_option('display.max_colwidth', 200)
df

,label,text
0,5,I want to travel back to my native place.
1,5,Want curfew pass for case of emergency travel.
2,5,I had started towards my home 120 km away on my car.
3,5,Firstline workers facing issues as no cabs are available
4,5,Want to go back to my family place. Currently staying in the hotel.
...,...,...
333,4,Economic downfall will soon create job crises.
334,4,Employee are not happy with the relief funds provided.
335,4,emi should be reduced and less interest rate should be charged on loans and other policies.
336,4,tv news channel focus more on fake news.


In [7]:
import random
data=[]
length=len(df)

for i in range(length * 10):
    num1 = i % length
    num2 = num1
    while (num1 == num2):
        num2=random.randrange(length)
    data.append([df.label[num1],df.label[num2],df.text[num1] + ' ' + df.text[num2]])
    
for i in range(length):
    data.append([df['label'][i], df['label'][i], df['text'][i]])     
df2 = pd.DataFrame(data, columns=['label1','label2', 'text'])
#df2.append(pd.DataFrame([df['label'], df['label'], df['text']]))

In [8]:
df2['label2'].value_counts()

3    815
0    750
2    676
5    646
4    436
1    395
Name: label2, dtype: int64

In [9]:
trainset.target_names

['communication', 'essential_items', 'food', 'healthcare', 'others', 'travel']

In [10]:
df2.tail(10)

,label1,label2,text
3708,4,4,She is an absolute arrogant disgrace - shameful and no respect for the gentleman she was interviewing. She planned to gang up on him and hit him hard - lack of rational sense was from her. Pur...
3709,4,4,Think there's a minority who have an issue with NHS staff as I got verbally abused in a store - a woman shouted in my face as she pushed me to go in a queue. When I brought my ID out to get disco...
3710,4,4,Took me years to realise. It isn't ok! If the Warning signs show get out from abuse!
3711,4,4,They lost their jobs due to COVID-19. Request you to pls help them.
3712,4,4,I lost my job due to this sudden covid outbreak.
3713,4,4,Economic downfall will soon create job crises.
3714,4,4,Employee are not happy with the relief funds provided.
3715,4,4,emi should be reduced and less interest rate should be charged on loans and other policies.
3716,4,4,tv news channel focus more on fake news.
3717,4,4,work from home should be manditory for all companies


In [11]:
# df2.to_csv('dataset_multi.csv')

In [12]:
import spacy
from nltk.stem import WordNetLemmatizer

In [13]:
nlp = spacy.load("en_core_web_lg")
filter_POS = set(['NOUN', 'NUM', 'ADJ', 'VERB', 'PART'])
lemmatizer = WordNetLemmatizer()
# ADV, NOUN, ADP, PRON, SCONJ, PROPN, DET, SYM, INTJ
# PUNCT, NUM, AUX, X, CONJ, ADJ, VERB, PART, SPACE, CCONJ
# def filter_query(query):
#     query = query.lower()
#     doc = nlp(query)
#     tokens = [t.text for t in doc if t.pos_ in filter_POS]
#     return ' '.join(tokens)
gc = 0
def filter_query(query):
    query = query.lower()
    query = re.sub(r'[\t\n\r\f ]+', ' ', re.sub(r'\.', '. ', query))
    # print (query)
    doc = nlp(query)
    if len(doc) > 6:
        tokens = [t.text for t in doc if t.pos_ in filter_POS]
    else:
        tokens = [t.text for t in doc]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    filt_q = ' '.join(tokens)
    filt_q = re.sub(r'\b(n\'t|nt)\b', 'not', filt_q)
    filt_q = re.sub(r'\'ll\b', 'will', filt_q)
    return filt_q

In [14]:
df2['text'].shape

(3718,)

In [15]:
df2['text_'] = df2['text']
df2['text'] = df2['text'].apply(filter_query)

# df['text_'] = df['text']
# df['text'] = df['text'].apply(filter_query)

In [16]:
# df2_ = df2.copy()

In [17]:
df2

,label1,label2,text,text_
0,5,1,want to travel native place toilet paper bar soap liquid soap toothpaste facial tissue shampoo,I want to travel back to my native place. toilet paper bar soap liquid soap toothpaste facial tissue shampoo
1,5,0,want curfew pas case emergency travel want to ask pending two month electricity bill fir fund issue not able to pay bill let know not make payment will disconnect line request to please wevied ine...,Want curfew pass for case of emergency travel. I want to ask i have pending two month electricity bill..fir to covid and fund issue not able to pay bill... Please let me know if I do not make paym...
2,5,3,started home 120 km car adequate stockpile necessary medical supply type contingency known generation situation arise capitalism not safeguard public health,I had started towards my home 120 km away on my car. There have never been adequate stockpiles of necessary medical supplies for this type of contingency even though it's been known for generation...
3,5,4,firstline worker facing issue cab available reduce oil price,Firstline workers facing issues as no cabs are available Reduce oil prices immidiately
4,5,3,want to go family place staying hotel food poisoning eating old distributed item,Want to go back to my family place. Currently staying in the hotel. food poisoning by eating old distributed items
...,...,...,...,...
3713,4,4,economic downfall will create job crisis,Economic downfall will soon create job crises.
3714,4,4,employee not happy relief fund provided,Employee are not happy with the relief funds provided.
3715,4,4,should reduced le interest rate should charged loan other policy,emi should be reduced and less interest rate should be charged on loans and other policies.
3716,4,4,focus fake news,tv news channel focus more on fake news.


In [18]:
X_train, X_test, y_train, y_test = train_test_split(df2, df2['label2'], test_size=0.2)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)


# X_train.reset_index(drop=True)
# X_test.reset_index(drop=True)
# y_train.reset_index(drop=True)
# y_test.reset_index(drop=True)

In [19]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

main=[]
for i in range(len(X_train)):
    arr=np.array([X_train['label1'][i],X_train['label2'][i]])
    main.append(arr)
    
# main=[]
# for i in range(len(X_train)):
#     arr=np.array([X_train['label'][i]])
#     main.append(arr)
    
# # Create MultiLabelBinarizer object
# one_hot = MultiLabelBinarizer()

# One-hot encode data
train_text = np.array(X_train, dtype=object)[:,2][:,np.newaxis]
# train_label =np.asarray(one_hot.fit_transform(main), dtype = np.int8)

train_label = np.zeros((X_train.shape[0], len(trainset.target)))
for i in range(X_train.shape[0]):
    train_label[i][main[i]] = 1

In [20]:
train_label[9], X_train.loc[9]

(array([0., 0., 1., 1., 0., 0.]),
 label1                                                                                                                                                                                                          3
 label2                                                                                                                                                                                                          2
 text                                     travelling met accident need medical assistance fractured leg today organized virtual event food security nutrition looking response early result to avert global crisis
 text_     I was travelling from karnataka to tamil nadu and met with an accident. I need medical assistance for my fractured leg. Today we organized a virtual event on food security and nutrition looking at...
 Name: 9, dtype: object)

In [21]:
main[10],train_label[10], train_text[10]

(array([1, 1], dtype=int64),
 array([0., 1., 0., 0., 0., 0.]),
 array(['number essential item bandaids gauze inhaler white sheet required 50 people'],
       dtype=object))

In [22]:
len(trainset.target)

6

In [23]:
main=[]
for i in range(len(X_test)):
    arr=np.array([X_test['label1'][i],X_test['label2'][i]])
    main.append(arr)
    
# main=[]
# for i in range(len(X_test)):
#     arr=np.array([X_test['label'][i]])
#     main.append(arr)
    
# # Create MultiLabelBinarizer object
# one_hot = MultiLabelBinarizer()

test_text = np.array(X_test, dtype=object)[:,2][:,np.newaxis]
# test_label = np.asarray(one_hot.fit_transform(main), dtype = np.int8)

test_label = np.zeros((X_test.shape[0], len(trainset.target)))
for i in range(X_test.shape[0]):
    test_label[i][main[i]] = 1

In [28]:
train_text_use = train_text[:, 0]
test_text_use = test_text[:, 0]
print (train_text_use)

['week network ballons went dead thyroiditis type autoimmune thyroid disease immune system attack change texture thyroid gland'
 'edible supply not available area need bread grocery day student learning to bully school'
 'need to take father hospital dialysis sparking electric generator fuel leakage got third degree burn'
 ... '5 people trapped internet locality'
 '6 people family food can give want to go family place staying hotel'
 'wife construction worker not enough meal due shortage money weak unable to water storage container bulb tubelights battery charger plug required']


In [24]:
category_counts = len(np.unique(df.label))
module_url = "../tf_sent_encoder_2" 
embed = hub.Module(module_url, trainable=True)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [175]:
categories = trainset.target_names
categories

# from keras.layers import Concatenate

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]

# def sentence_encoder(input_text):
#     return embed(tf.squeeze(input_text))

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(1024, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = []
for c in categories:
    pred.append(Dense(1, activation='sigmoid')(dense2))
out = Concatenate()(pred)
model = Model(inputs=[input_text], outputs=out)

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


KeyboardInterrupt: 

In [131]:
# def create_model(vocab_size, max_length):
#     model = Sequential()

#     model.add(Embedding(vocab_size, 128,
#             input_length = max_length,  trainable = False))
#     model.add(Bidirectional(GRU(128)))
#     model.add(Dense(64, activation = "relu"))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation = "relu"))
#     model.add(Dropout(0.5))
#     model.add(BatchNormalization())
#     model.add(Dense(21, activation = "softmax"))

#     return model
print (K.eval(model.optimizer.lr))
K.set_value(model.optimizer.lr, 0.0005)
print (K.eval(model.optimizer.lr))

0.001
0.0005


In [41]:
# tf.initialize_all_variables().run(session=tf.keras.backend.get_session())
hub.__version__

'0.8.0'

In [40]:
K.set_session(session)
class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(" - lr: {}".format(K.eval(self.model.optimizer.lr))) 
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

LR_PATIENCE = 10
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=LR_PATIENCE, min_lr=1e-8, verbose=1, mode="min")
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
lr_tracker = LearningRateTracker()

history = model.fit(train_text_use, 
          train_label,
          validation_data=(test_text_use, test_label),
          epochs=1000,
          batch_size=256,
          callbacks=[es_callback, lr_tracker, reduce_lr])
model.save_weights('./model20.h5')

Train on 2974 samples, validate on 744 samples
Epoch 1/1000


FailedPreconditionError: [_Derived_]{{function_node __inference_pruned_9755}} {{function_node __inference_pruned_9755}} Attempting to use uninitialized value Embeddings_en/sharded_1
	 [[{{node Embeddings_en/sharded_1/read}}]]
	 [[keras_layer/StatefulPartitionedCall]]

In [133]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               131200    
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 774       
Total params: 657,286
Trainable params: 657,286
Non-trainable params: 0
_________________________________________________________________


In [68]:
# def ElmoEmbeddingLayer(x):
# #     print(x.shape)
#     module = hub.Module("../nnlm-en-dim128_1", trainable=True)
#     embeddings = module(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
#     return embeddings
nnlm_dim=128
nnlm_input = Input(shape=(), dtype=tf.string)
# nnlm_embedding = Lambda(ElmoEmbeddingLayer, output_shape=(nnlm_dim,))(nnlm_input)
nnlm_embedding = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2", output_shape=[128], input_shape=[], dtype=tf.string, trainable=False)(nnlm_input)
x = Dense(512, activation='relu')(nnlm_embedding)
x = Dense(512, activation='relu')(x)
out = Dense(len(categories), activation='sigmoid')(x)

model = Model(inputs=[nnlm_input], outputs=out)
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.fit(x_train, y_train, epochs=1,validation_data=(x_test, y_test))


LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

In [69]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
keras_layer_6 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_6 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 3078      
Total params: 124,974,470
Trainable params: 331,782
Non-trainable params: 124,642,688
_________________________________________________________________


In [58]:
# K.set_session(session)
class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(" - lr: {}".format(K.eval(self.model.optimizer.lr))) 
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

LR_PATIENCE = 10
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=LR_PATIENCE, min_lr=1e-8, verbose=1, mode="min")
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
lr_tracker = LearningRateTracker()

history = model.fit(train_text_use, 
          train_label,
          validation_data=(test_text_use, test_label),
          epochs=1000,
          batch_size=256,
          callbacks=[es_callback, lr_tracker, reduce_lr])
model.save_weights('./model20.h5')

Train on 2974 samples, validate on 744 samples
Epoch 1/1000
2974/2974 [==============================] - 7s 2ms/step - loss: 0.5890 - acc: 0.6899 - val_loss: 0.5127 - val_acc: 0.7507
 - lr: 0.0010000000474974513
Epoch 2/1000
2974/2974 [==============================] - 1s 206us/step - loss: 0.4466 - acc: 0.8121 - val_loss: 0.3823 - val_acc: 0.8434
 - lr: 0.0010000000474974513
Epoch 3/1000
2974/2974 [==============================] - 1s 181us/step - loss: 0.3299 - acc: 0.8677 - val_loss: 0.3035 - val_acc: 0.8853
 - lr: 0.0010000000474974513
Epoch 4/1000
2974/2974 [==============================] - 1s 174us/step - loss: 0.2647 - acc: 0.8976 - val_loss: 0.2601 - val_acc: 0.8985
 - lr: 0.0010000000474974513
Epoch 5/1000
2974/2974 [==============================] - 0s 163us/step - loss: 0.2278 - acc: 0.9110 - val_loss: 0.2360 - val_acc: 0.9140
 - lr: 0.0010000000474974513
Epoch 6/1000
2974/2974 [==============================] - 1s 182us/step - loss: 0.2045 - acc: 0.9206 - val_loss: 0.2184 

2974/2974 [==============================] - 1s 302us/step - loss: 0.0111 - acc: 0.9992 - val_loss: 0.0762 - val_acc: 0.9738
 - lr: 0.0010000000474974513
Epoch 50/1000
2974/2974 [==============================] - 1s 270us/step - loss: 0.0108 - acc: 0.9991 - val_loss: 0.0737 - val_acc: 0.9747
 - lr: 0.0010000000474974513
Epoch 51/1000
2974/2974 [==============================] - 1s 266us/step - loss: 0.0104 - acc: 0.9993 - val_loss: 0.0738 - val_acc: 0.9749
 - lr: 0.0010000000474974513
Epoch 52/1000
2974/2974 [==============================] - 1s 264us/step - loss: 0.0096 - acc: 0.9992 - val_loss: 0.0740 - val_acc: 0.9760
 - lr: 0.0010000000474974513
Epoch 53/1000
2974/2974 [==============================] - 1s 252us/step - loss: 0.0093 - acc: 0.9991 - val_loss: 0.0745 - val_acc: 0.9747
 - lr: 0.0010000000474974513
Epoch 54/1000
2974/2974 [==============================] - 1s 258us/step - loss: 0.0088 - acc: 0.9993 - val_loss: 0.0729 - val_acc: 0.9751
 - lr: 0.0010000000474974513
Epoch 

2974/2974 [==============================] - 1s 275us/step - loss: 0.0026 - acc: 0.9996 - val_loss: 0.0693 - val_acc: 0.9778
 - lr: 0.0002500000118743628
Epoch 97/1000
2974/2974 [==============================] - 1s 262us/step - loss: 0.0027 - acc: 0.9994 - val_loss: 0.0695 - val_acc: 0.9776
 - lr: 0.0002500000118743628
Epoch 98/1000
2974/2974 [==============================] - 1s 267us/step - loss: 0.0027 - acc: 0.9994 - val_loss: 0.0693 - val_acc: 0.9785
 - lr: 0.0002500000118743628

Epoch 00098: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 99/1000
2974/2974 [==============================] - 1s 254us/step - loss: 0.0026 - acc: 0.9993 - val_loss: 0.0696 - val_acc: 0.9787
 - lr: 0.0001250000059371814
Epoch 100/1000
2974/2974 [==============================] - 1s 261us/step - loss: 0.0026 - acc: 0.9996 - val_loss: 0.0696 - val_acc: 0.9787
 - lr: 0.0001250000059371814
Epoch 101/1000
2974/2974 [==============================] - 1s 255us/step - loss: 0.0025 - a

In [70]:
model.load_weights('model_nnlm128_512_512.h5')

In [120]:
import csv
queries_list = []
with open('COVID-TEXT.csv', 'r', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        queries_list.append(row[0])

In [121]:
queries_list

['Labourers without food',
 'Migrant Workers from UP and BIHAR',
 'People need cooked food',
 'people stuck in bangalore',
 'People Stuck without Food',
 'People without ration',
 'Residence of rama swamy palya facing water scarcity',
 'Stranded migrants from various states',
 '5o Migrant workers from West Bengal',
 'Alemari samudaya with food and ration',
 'Elder people need urgent ration',
 'Familiy without ration',
 '10 members need Ration , Daily Wage workers',
 '10 people Need information on how to go back to bihar ',
 '10 person from Bihar are struck in Dharwad. They need Ration , Kindly deliver, they have no money and food to survive .\nThanks',
 '14 Workers need ration',
 '150 people Wants to go home from mangalore to Giridih Jarkhand',
 '17 families are not having ration card not received any ration from anywhere. They need to be helped for the basic food and sanitaries',
 '25 people who are living in Venkateshwar Nagar, (Vaddargalli) not ration card and not benefiting from PD

In [ ]:
comb = Con
model3

In [77]:
def run_prediction(query_list):
    filt_query = [filter_query(q) for q in query_list]
    print (filt_query)
    # filt_query = query_list
    # query_arr = np.array(filt_query, dtype=object)[:, np.newaxis]
    query_arr = np.array(filt_query)
    predicts = model.predict(query_arr)
    return predicts

In [78]:
new_text = ["No network connectivity in my area for ordering groceries. Need ration urgently",
"Need gas cylinder as my mobile broke down.",
"My father is sick. i want daily ration",
"Yesterday i saw few people bullying a person from my locality. I think he needs help.i need cab to go to hospital.",
'I am unable to find fruits and vegetables',
'need food and travel for 50 families. contact number 8134981349',
'Me and my parents are stuck alone in our house with severe health issues. My father has heart problems and my mother is bedridden. I need urgent medical assistance']


# new_text = np.array(new_text, dtype=object)[:, np.newaxis]
# #new_text = np.array(queries_list, dtype=object)[:, np.newaxis]

# predicts = model.predict(new_text)
predicts = run_prediction(new_text)

dictt=trainset.target_names

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

['network connectivity area ordering grocery need ration', 'need gas cylinder mobile broke', 'father sick want daily ration', 'yesterday saw few people bullying person locality think need help need cab to go hospital', 'unable to find fruit vegetable', 'need food travel 50 family contact number 8134981349', 'parent stuck house severe health issue father heart problem mother bedridden need urgent medical assistance']
QUERY 0-> No network connectivity in my area for ordering groceries. Need ration urgently


	communication:100.0
	essential_items:0.1
	food:100.0
	healthcare:0.0
	others:0.0
	travel:0.0


QUERY 1-> Need gas cylinder as my mobile broke down.


	communication:0.0
	essential_items:100.0
	food:0.0
	healthcare:0.3
	others:0.3
	travel:0.2


QUERY 2-> My father is sick. i want daily ration


	communication:0.0
	essential_items:0.0
	food:100.0
	healthcare:90.7
	others:0.0
	travel:43.6


QUERY 3-> Yesterday i saw few people bullying a person from my locality. I think he needs help.i

In [82]:
new_text = ["No network connectivity in my area for order grocery . Need ration urgently",

 "i am try to contact airport authorities but they are not responding",

 "Yesterday i saw few people bullying a person from my locality. I think he needs help.i need cab as my car broke down to go to hospital.",

 'give me toilet cleaner and pet food',

 'ijust travel back to bangalore due to lockdown i am not able to find soap',

 'i want ventilators',

 'need to go home urgently.',

 'Me and my parents are stuck alone in our house with severe health issues. My father has heart problems and my mother is bedridden. I need urgent medical assistance']
predicts = run_prediction(new_text)

dictt=trainset.target_names

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

['network connectivity area order grocery need ration', 'try to contact airport authority not responding', 'yesterday saw few people bullying person locality think need help need cab car broke to go hospital', 'give toilet cleaner pet food', 'travel lockdown not able to find soap', 'i want ventilator', 'need to go home urgently .', 'parent stuck house severe health issue father heart problem mother bedridden need urgent medical assistance']
QUERY 0-> No network connectivity in my area for order grocery . Need ration urgently


	communication:100.0
	essential_items:0.0
	food:100.0
	healthcare:0.0
	others:0.0
	travel:0.0


QUERY 1-> i am try to contact airport authorities but they are not responding


	communication:100.0
	essential_items:0.2
	food:0.1
	healthcare:0.0
	others:0.0
	travel:12.3


QUERY 2-> Yesterday i saw few people bullying a person from my locality. I think he needs help.i need cab as my car broke down to go to hospital.


	communication:0.0
	essential_items:0.0
	food:0.

In [56]:
model.predict(np.array(['need to go home', ''], dtype=object)[:, np.newaxis])

array([[6.0719925e-05, 5.1792540e-02, 3.0541523e-05, 3.5929384e-05,
        7.5689690e-07, 3.2593703e-01],
       [8.1119651e-05, 9.8318785e-01, 2.1166863e-06, 1.6125040e-02,
        6.0552239e-02, 5.8824737e-03]], dtype=float32)

In [107]:
filter_query("i want roti ,  egg , dal , chawal , anda .")

[('i', 'PRON'), ('want', 'VERB'), ('roti', 'NOUN'), (',', 'PUNCT'), ('egg', 'NOUN'), (',', 'PUNCT'), ('dal', 'PROPN'), (',', 'PUNCT'), ('chawal', 'PROPN'), (',', 'PUNCT'), ('anda', 'PROPN'), ('.', 'PUNCT')]


'want roti egg'

In [39]:
result_df = pd.DataFrame([[queries_list[i]] + list(np.around(predicts*100,decimals=1)[i]) for i in range(len(queries_list))], columns=['query'] + dictt)

IndexError: index 7 is out of bounds for axis 0 with size 7

In [50]:
result_df.head()

,query,communication,food,healthcare,others,travel
0,Labourers without food,0.000000,100.0,0.000000,0.800000,0.000000
1,Migrant Workers from UP and BIHAR,0.000000,0.6,48.900002,0.100000,99.099998
2,People need cooked food,0.300000,100.0,0.200000,64.699997,0.000000
3,people stuck in bangalore,98.099998,0.0,0.000000,100.000000,0.000000
4,People Stuck without Food,14.600000,100.0,0.000000,8.600000,0.000000


In [51]:
result_df.to_csv('results.csv', index=False)

In [24]:
query_arr = np.array(['need to visit home', ''], dtype=object)[:, np.newaxis]
model.predict(query_arr)

array([[2.2256071e-07, 1.9670317e-04, 2.2590014e-03, 4.9715425e-04,
        9.9989069e-01],
       [3.3535997e-03, 5.9001707e-03, 6.5540969e-01, 1.1426200e-02,
        2.5253209e-01]], dtype=float32)

In [169]:
new_text = ["We have a plea from Sunkadkatte for rations. Request your urgent help .They receive food sometimes but not regularly. They have children and would prefer ration.",
"There is a Dalit village with 150 families in Kanakapura district who are not receiving any support. They received 10 kg rice and some wheat per family at the start of the lockdown.Anyone can help ?",
"Dear Ma'am Suchitra here again with another request for help. About 10 people and two children, tribals from Javadhi hills, Tamil Nadu working in coffee n pepper plantation in Chikmagalur. 3 of them spoke to me. They say over 150+ such labourers are there in this belt. They were paid by the contractor in March n have no money now. They have not been given any ration either. They met the Tasildar yesterday about going back. But that cannot happen now.  Ma'am how do we help them out? What is the state decision on migrant workers moving back to place of origin? Thank you.",
"Sir these wkrs r working as carpenters, helpers at Arabhikothanur Kolar,the builder and contractors r providing groceries and other food requirements to these workers.on enquiry the wkrs told there is no problem with resp to food sir",
"250 Fishermen From Andhra Pradesh struck in Udupi, Karnataka.They didn't have access to food from last few days. They are in depression.And at last he says, ""agar Kuch ni mila toh zehar kha lenge isse aacha .Please help.",
"Dear friends,  08 members from Uttar Pradesh are  in Mangalore near Kannanuru check post, they are in critical situation present they Requesting food and other help, any organizations working in Mangalore and border of Kerala request you pls attend this cae and help them, all are construction migrant workers,  their contact No: 6282662916, I hope some one will reach them. Please provide ration for all of us.",
"These are the contact details of a group of 50 plus migrant workers stuck in KR market with only intermittent access to food. They say they are a large group and all of them are unable to get food even at distribution points as food runs out. Request you to help",
'We are a small village on the outskirts of Bangalore and we do not have good cell reception here and hence we are isolated here with no news about coronavirus or availability of supplies.',
'Three construction workers from jharkhand are hungry from last two days and jumped off the building. Please provide ambulance',
'my friend fell from terrace while searching for network. There is network problem in this area. This is all government mistake and the medical expenses should be bourne by them.',
'The daily ration kit distributed to us do not have cooking oil in them. How are we supposed to cook.',
"I am trying to start a small kitchen to feed the hungry poor kids in my locality but I am unable to do so as I haven't received the big 19 kg gas cannister which I applied for almost 20 days ago.",
'I am a doctor, my car brokedown and want it to get repaired urgently.']
# new_text = np.array(new_text, dtype=object)[:, np.newaxis]


# predicts = model.predict(new_text)


# categories = np.unique(df.label)
# predict_logits = predicts.argmax(axis=1)
# predict_labels = [categories[logit] for logit in predict_logits]
predicts = run_prediction(new_text)

dictt=trainset.target_names
# ANS=[dict[s] for s in predict_labels]

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

['plea sunkadkatte ration request urgent help receive food not child would prefer ration', 'dalit village 150 family not receiving support received 10 kg rice wheat family start lockdown can help', 'request help 10 people two child tribals working coffee pepper plantation 3 spoke say 150 such labourer belt paid contractor money not given ration met tasildar yesterday going can not happen help state decision migrant worker moving place origin thank', 'r working carpenter helper builder contractor r providing grocery other food requirement worker enquiry told problem food sir', '250 fisherman struck not access food last few day depression say help', 'dear friend 08 member check post critical situation present requesting food other help organization working border request attend cae help construction migrant worker contact 6282662916 hope one will reach provide ration', 'contact detail group 50 migrant worker stuck kr market only intermittent access food say large group unable to food dis

In [170]:
new_text = ["brush toothpaste and soap for 2 people and 50 units of n95 mask",
            "brush toothpaste and soap and 50 units of n95 mask",
           "rice, wheat and daal for 2 people and 50 units of n95 mask and syrup",
           "And the second is: Sri Venkateswara Layout, Chinappa Layout, Mahadevapura. There are 350 workers- construction, recycling, domestic help. WB, UP, Bihar. need urgent dry ration",
           "There are 350 workers- construction, recycling, domestic help. WB, UP, Bihar. need urgent dry ration",
           "et rations from a govt distribution in Sunkadkatte they didn't give him saying he is not from here. So request help here in this case too since they are not locals",
           '''Respected Ma'am,
Help Required.
No ration since yesterday who are in really need of food .
Pa Shehgin & family including 2 children.
1. Pa. Shehgin kipgen( worked in sales )
2. Nu. Hoineng kipgen( worked beauty salon )
3. Chingnu( daughter )
4. Dedai( daughter )
5. Gugun( son)
6. Hekham
7. Hoineo
8. Chinkho
Address:
#12, 19 A cross Lakshmipuram, Halasuru, Bengaluru 560008.
Contact number : 8884356927
1.Mrs.hahat doungel ( working in beauty salon) 2.boichung doungel ( son,7 year old )
3.jenny.doungel ( sister ).
Contact no : 9108396170.
Address: Kothanur, Anjinapalia, 3 cross''',
           "intrested in donating used blankets",
           "intrested in used blankets"
]
# new_text = np.array(new_text, dtype=object)[:, np.newaxis]


# predicts = model.predict(new_text)


# categories = np.unique(df.label)
# predict_logits = predicts.argmax(axis=1)
# predict_labels = [categories[logit] for logit in predict_logits]
predicts = run_prediction(new_text)

dictt=trainset.target_names
# ANS=[dict[s] for s in predict_labels]

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')

['brush toothpaste soap 2 people 50 unit mask', 'brush toothpaste soap 50 unit mask', 'rice wheat daal 2 people 50 unit mask syrup', 'second layout 350 workers- construction recycling domestic help need urgent dry ration', '350 workers- construction recycling domestic help need urgent dry ration', 'ration govt distribution sunkadkatte not give saying not request help case not local', 'respected help required ration yesterday need food family including 2 child 1 worked sale 2 worked beauty salon 3 daughter 4 daughter 5 6 7 8 address 12 19 cross 560008 contact number 8884356927 1 working beauty salon 2 year old sister contact 9108396170 address 3 cross', 'intrested in donating used blanket', 'intrested in used blanket']
QUERY 0-> brush toothpaste and soap for 2 people and 50 units of n95 mask


	communication:0.0
	essential_items:99.3
	food:0.9
	healthcare:84.6
	others:2.5
	travel:0.0


QUERY 1-> brush toothpaste and soap and 50 units of n95 mask


	communication:0.0
	essential_items:99.

In [ ]:
["We have a plea from Sunkadkatte for rations. Request your urgent help .They receive food sometimes but not regularly. They have children and would prefer ration.",
"There is a Dalit village with 150 families in Kanakapura district who are not receiving any support. They received 10 kg rice and some wheat per family at the start of the lockdown.Anyone can help ?",
"Dear Ma'am Suchitra here again with another request for help. About 10 people and two children, tribals from Javadhi hills, Tamil Nadu working in coffee n pepper plantation in Chikmagalur. 3 of them spoke to me. They say over 150+ such labourers are there in this belt. They were paid by the contractor in March n have no money now. They have not been given any ration either. They met the Tasildar yesterday about going back. But that cannot happen now.  Ma'am how do we help them out? What is the state decision on migrant workers moving back to place of origin? Thank you.",
"Sir these wkrs r working as carpenters, helpers at Arabhikothanur Kolar,the builder and contractors r providing groceries and other food requirements to these workers.on enquiry the wkrs told there is no problem with resp to food sir",
"250 Fishermen From Andhra Pradesh struck in Udupi, Karnataka.They didn't have access to food from last few days. They are in depression.And at last he says, ""agar Kuch ni mila toh zehar kha lenge isse aacha .Please help.",
"Dear friends,  08 members from Uttar Pradesh are  in Mangalore near Kannanuru check post, they are in critical situation present they Requesting food and other help, any organizations working in Mangalore and border of Kerala request you pls attend this cae and help them, all are construction migrant workers,  their contact No: 6282662916, I hope some one will reach them.",
"These are the contact details of a group of 50 plus migrant workers stuck in KR market with only intermittent access to food. They say they are a large group and all of them are unable to get food even at distribution points as food runs out. Request you to help"]

In [33]:
predicts.shape
preds_round = np.around(predicts*100,decimals=1)

In [69]:
from statistics import stdev
pred_results = []
for i in range(predicts.shape[0]):
    sorted_class_similariy = [float(x) for x in sorted(preds_round[i], reverse=True)]
    print (sorted_class_similariy)
    split_stdev = []
    split_stdev.append(stdev(sorted_class_similariy[1:len(dictt)]))
    for k in range(2, len(dictt) - 1):
        split_stdev.append((stdev(sorted_class_similariy[:k]) + stdev(sorted_class_similariy[k:]))/2)
    split_stdev.append(stdev(sorted_class_similariy[:len(dictt) - 1]))

    split_ind = np.argmin(split_stdev)
    thres = (sorted_class_similariy[split_ind] + sorted_class_similariy[split_ind + 1]) / 2
    # print (thres)
    pred_labels = [c for c in range(len(dictt)) if preds_round[i][c] >= np.around(thres, decimals=1)]
    pred_classes = [dictt[r] for r in pred_labels]
    # print (pred_labels)
    pred_results.append(pred_classes)

[100.0, 92.9000015258789, 0.5, 0.0, 0.0, 0.0]
[100.0, 6.800000190734863, 5.5, 0.20000000298023224, 0.0, 0.0]
[99.9000015258789, 83.80000305175781, 56.900001525878906, 0.0, 0.0, 0.0]
[94.80000305175781, 93.80000305175781, 35.099998474121094, 2.200000047683716, 0.8999999761581421, 0.0]
[100.0, 70.9000015258789, 0.5, 0.0, 0.0, 0.0]
[100.0, 71.9000015258789, 0.0, 0.0, 0.0, 0.0]
[97.80000305175781, 41.79999923706055, 20.0, 10.0, 0.0, 0.0]
[100.0, 99.9000015258789, 3.200000047683716, 0.30000001192092896, 0.10000000149011612, 0.0]
[100.0, 4.199999809265137, 2.4000000953674316, 0.6000000238418579, 0.20000000298023224, 0.0]
[100.0, 100.0, 0.10000000149011612, 0.10000000149011612, 0.0, 0.0]
[100.0, 51.099998474121094, 2.299999952316284, 0.0, 0.0, 0.0]
[100.0, 27.5, 0.20000000298023224, 0.0, 0.0, 0.0]
[100.0, 2.200000047683716, 0.4000000059604645, 0.4000000059604645, 0.20000000298023224, 0.0]
[99.9000015258789, 6.5, 3.4000000953674316, 0.5, 0.20000000298023224, 0.10000000149011612]
[99.8000030517

In [70]:
pred_results

[['food', 'health'],
 ['food'],
 ['food', 'fuel', 'health'],
 ['communication', 'travel'],
 ['food', 'health'],
 ['food', 'health'],
 ['travel'],
 ['food', 'travel'],
 ['food'],
 ['food', 'health'],
 ['abuse', 'food'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['communication', 'food'],
 ['food', 'travel'],
 ['food', 'health', 'travel'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'health'],
 ['food'],
 ['travel'],
 ['food', 'health', 'travel'],
 ['food', 'fuel', 'travel'],
 ['health'],
 ['food'],
 ['travel'],
 ['food'],
 ['food'],
 ['food'],
 ['food', 'health'],
 ['food'],
 ['travel'],
 ['communication', 'travel'],
 ['food'],
 ['food'],
 ['food', 'travel'],
 ['food'],
 ['food', 'travel'],
 ['food', 'fuel', 'health'],
 ['food', 'fuel', 'travel'],
 ['fuel'],
 ['food', 'fuel', 'travel'],
 ['food'],
 ['food'],
 ['abuse'],
 ['food', 'fuel'],
 ['travel'],
 ['food'],
 ['food', 'travel'],
 ['abuse', 'food'],
 ['abuse', 'fuel', 

In [ ]:
OK### both tanh -84% split=0.3    split=o.2 good
## both relu - 86% split=0.2
## both tanh - 85.4% split=0.2
## tanh relu 84.73
### relu tanh 85.63 -----good 256,128
### relu tanh  86.12-----ok ok 1024,256
### relu tanh  86.12-----ok ok 1024,256
## tanh tanh ---1024 128 
## relu tanh tanh 256 128 64- 85.8%
## tanh tanh 0.3 split acc= 84.55 can try


OK###with categorical cross entropy tanh tanh 256 128 results are ok but accuracy showing is less




### 93.82%
# dense1 = Dense(256, activation='relu')(embedding)
# dense2 = Dense(128, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])




### 94.41%
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(256, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])



## __94.78____%
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(128, activation='relu')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])



####### 94.48 % #########

# input_text = Input(shape=(1,), dtype=tf.string)
# embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(128, activation='tanh')(dense1)
# pred = Dense(category_counts, activation='sigmoid')(dense2)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [161]:

# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

In [162]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [157]:
# elmo = hub.Module('../elmo_2', trainable=True)
# elmo(K.squeeze(K.cast([['food'],['']], tf.string), axis=1),
#                         as_dict=True,
#                         signature='default',
#                         )['default']

In [140]:
# model_url = "https://tfhub.dev/google/elmo/3"
# # hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
# hub_layer = hub.KerasLayer(model_url, output_shape=[128], input_shape=[], 
#                            dtype=tf.string, trainable=False, 
#                            signature="default", output_key="default")
# hub_layer(train_examples[:3])
elmo_emb_layer = ElmoEmbeddingLayer().build((None,))
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(len(categories), activation='sigmoid'))
model.build(input_shape=())
model.summary()



# dense1 = Dense(1024, activation='relu')(embedding)
# dense2 = Dense(128, activation='relu')(dense1)
# out = Dense(len(categories), activation='sigmoid')(dense2)
# # pred = []
# # for c in categories:
# #     pred.append(Dense(1, activation='sigmoid')(dense2))
# # out = Concatenate()(pred)
# model = Model(inputs=[input_text], outputs=out)

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


NameError: name 's' is not defined

In [172]:
def build_model(): 
    input_text = layers.Input(shape=(1,), dtype="string")
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(6, activation='sigmoid')(dense)

    model = Model(inputs=[input_text], outputs=pred)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

model = build_model()
# model.fit(train_text, 
#           train_label,
#           validation_data=(test_text, test_label),
#           epochs=10,
#           batch_size=32)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_19 (Elm (None, 1024)              4         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 1542      
Total params: 263,946
Trainable params: 263,946
Non-trainable params: 0
_________________________________________________________________


In [173]:
model.fit(train_text, 
          train_label,
          validation_data=(test_text, test_label),
          epochs=5,
          batch_size=256)


Train on 2974 samples, validate on 744 samples
Epoch 1/5
2974/2974 [==============================] - 818s 275ms/step - loss: 0.5129 - acc: 0.7512 - val_loss: 0.3913 - val_acc: 0.8566
Epoch 2/5
2974/2974 [==============================] - 800s 269ms/step - loss: 0.3262 - acc: 0.8754 - val_loss: 0.2885 - val_acc: 0.8849
Epoch 3/5
2974/2974 [==============================] - 917s 308ms/step - loss: 0.2510 - acc: 0.9069 - val_loss: 0.2440 - val_acc: 0.9017
Epoch 4/5
2974/2974 [==============================] - 901s 303ms/step - loss: 0.2106 - acc: 0.9205 - val_loss: 0.2099 - val_acc: 0.9216
Epoch 5/5
2974/2974 [==============================] - 878s 295ms/step - loss: 0.1819 - acc: 0.9322 - val_loss: 0.1884 - val_acc: 0.9323


In [176]:
model.save_weights('model_elmo_256_1.h5')